In [72]:
import requests
import re
import uuid
from bs4 import BeautifulSoup

In [26]:
url = "https://vinepair.com/cocktail-recipe/?fwp_paged=1"
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
}
reqs = requests.get(url, headers=headers)
soup = BeautifulSoup(reqs.text, 'html.parser')
#print(reqs.text)

In [36]:
recipe_pattern = re.compile(".*cocktail-recipe/.+")

In [38]:
recipe_links = []
for link in soup.find_all('a'):
    valid_link = link.get('href')
    if valid_link == None:
        continue
    if recipe_pattern.match(valid_link) and valid_link not in recipe_links:
        recipe_links.append(valid_link)

In [43]:
def scrape_recipe_page(url, link_store):
    headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    }
    reqs = requests.get(url, headers=headers)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    recipe_pattern = re.compile(".*cocktail-recipe/.+")
    for link in soup.find_all('a'):
        valid_link = link.get('href')
        if valid_link == None:
            continue
        if recipe_pattern.match(valid_link) and valid_link not in link_store:
            link_store.append(valid_link)
    return link_store

In [56]:
url = "https://vinepair.com/cocktail-recipe/paloma/"
reqs = requests.get(url, headers=headers)
soup = BeautifulSoup(reqs.text, 'html.parser')

In [94]:
out_yaml = open('paloma.yaml', 'a')

In [95]:
recipe_uuid = uuid.uuid4()
print(str(recipe_uuid))
out_yaml.write("recipe_uuid: " + str(recipe_uuid) + "\n")

ec081564-abac-4ee2-957d-ecfc7e4f9aec


50

In [96]:
postfix_pattern = re.compile(".* Recipe$")
for title in soup.find_all('h1', {"class": "entry-title"}):
    name = title.text
    if postfix_pattern.match(name):
        name = name[:-7]
    print(name)
    out_yaml.write("recipe_name: " + name + "\n")

Paloma


In [97]:
out_yaml.write("source_url: " + url + "\n")

57

In [98]:
for recipe_yield in soup.find_all('p', {"class": "review-extra-meta"}):
    print(recipe_yield.text)
    yield_data = recipe_yield.text.split()
    out_yaml.write("yields:\n  - amount: "+ yield_data[1] + "\n" + "    unit: " + yield_data[2] + "\n")


Yield: 1 Cocktail
Updated: 2016-04-14



In [99]:
out_yaml.write("ingredients:\n")
for ingredients in soup.find_all('li', {"class": "recipeIngredient"}):
    for span in ingredients.find_all('span'):
        print(span.text)
        if span == None:
            continue
        ingredient_data = span.text.split("\t")
        if len(ingredient_data) == 2:
            if ingredient_data[0] == "As needed":
                ingredient_data.insert(0, "1")
        out_yaml.write("  - " + ingredient_data[2] + 
                       ":\n      amounts:\n        - amount: " + 
                       ingredient_data[0] + 
                       "\n          unit: "  + 
                       ingredient_data[1] + "\n")

2 	oz	Reposado or Blanco tequila
1 	oz 	Lime
1	each	Pinch of salt
As needed	Grapefruit soda


In [100]:
out_yaml.write("steps:\n")
for steps in soup.find_all('ol', {"class": "recipeInstructionsList"}):
    for step in steps.find_all('li'):
        print(step.text)
        out_yaml.write("  - step:\n      " + step.text + "\n")

Combine ingredients in a highball.
Add ice.
Garnish with a grapefruit wheel and enjoy.


In [101]:
out_yaml.close()

In [168]:
def parse_recipe_to_yaml(url):
    reqs = requests.get(url, headers=headers)
    soup = BeautifulSoup(reqs.text, 'html.parser')
    
    postfix_pattern = re.compile(".* Recipe$")
    for title in soup.find_all('h1', {"class": "entry-title"}):
        name = title.text
        if postfix_pattern.match(name):
            name = name[:-7]
    out_yaml = open("cocktails/" + name.lower().replace(" ", "_") + ".yaml", 'a')
    
    recipe_uuid = uuid.uuid4()
    out_yaml.write("recipe_uuid: " + str(recipe_uuid) + "\n")
    
    out_yaml.write("recipe_name: " + name + "\n")
    
    out_yaml.write("source_url: " + url + "\n")
    
    for recipe_yield in soup.find_all('p', {"class": "review-extra-meta"}):
        lines = recipe_yield.text.split("\n")
        yield_data = lines[1].split()
        if len(yield_data) == 1:
            yield_data.append("1")
        if len(yield_data) == 2:
            yield_data.append("units")
        out_yaml.write("yields:\n  - amount: "+ yield_data[1] + "\n" + "    unit: " + yield_data[2] + "\n")
    
    out_yaml.write("ingredients:\n")
    for ingredients in soup.find_all('li', {"class": "recipeIngredient"}):
        for span in ingredients.find_all('span'):
            if span == None:
                continue
            #print(span.text)
            ingredient_data = span.text.split("\t")
            if len(ingredient_data) != 3:
                if len(ingredient_data) == 0:
                    continue
                if len(ingredient_data) == 1:
                    ingredient_data = span.text.split()
                    if len(ingredient_data) == 1:
                        ingredient_data.insert(0, "unit")
                        ingredient_data.insert(0, "1")
                try:
                    if ingredient_data[0] == "As needed":
                        ingredient_data.insert(0, "1")
                    elif ingredient_data[0] == "Garnish:":
                        ingredient_data[1] = " ".join(ingredient_data[1:])
                        ingredient_data[0] = "1"
                        ingredient_data.insert(1, "unit")
                        del ingredient_data[3:]
                    elif '/' in ingredient_data[1] or ingredient_data[1] in "½¼¾¾":
                        ingredient_data[0] = " ".join(ingredient_data[0:2])
                        del ingredient_data[1]
                    elif len(ingredient_data) == 2:
                        ingredient_data.insert(1, "unit")
                except IndexError:
                    continue
                if len(ingredient_data) > 3:
                    ingredient_data[2] = " ".join(ingredient_data[2:])
            out_yaml.write("  - " + ingredient_data[2] + 
                           ":\n      amounts:\n        - amount: " + 
                           ingredient_data[0] + 
                           "\n          unit: "  + 
                           ingredient_data[1] + "\n")
            
    out_yaml.write("steps:\n")
    for steps in soup.find_all('ol', {"class": "recipeInstructionsList"}):
        for step in steps.find_all('li'):
            out_yaml.write("  - step:\n      " + step.text + "\n")
    
    out_yaml.close()

In [134]:
recipe_links = []
print("Scraping recipe links from website...")
for i in range(1, 35):
    url = "https://vinepair.com/cocktail-recipe/?fwp_paged=" + str(i)
    recipe_links = scrape_recipe_page(url, recipe_links)

Scraping recipe links from website...


In [169]:
for link in recipe_links:
    print("Parsing " + link + "...")
    parse_recipe_to_yaml(link)

Parsing https://vinepair.com/cocktail-recipe/monkey-gland-recipe/...
Parsing https://vinepair.com/cocktail-recipe/tequila-hot-toddy/...
Parsing https://vinepair.com/cocktail-recipe/oaxaca-old-fashioned-recipe/...
Parsing https://vinepair.com/cocktail-recipe/peppermint-old-fashioned/...
Parsing https://vinepair.com/cocktail-recipe/poinsettia-old-fashioned/...
Parsing https://vinepair.com/cocktail-recipe/the-bees-knees-recipe/...
Parsing https://vinepair.com/cocktail-recipe/crown-royal-maple-apple/...
Parsing https://vinepair.com/cocktail-recipe/whiskey-highball-recipe/...
Parsing https://vinepair.com/cocktail-recipe/johnnie-walker-perfect-pear/...
Parsing https://vinepair.com/cocktail-recipe/blood-orange-paloma/...
Parsing https://vinepair.com/cocktail-recipe/ketel-one-classic-martini/...
Parsing https://vinepair.com/cocktail-recipe/don-julio-spicy-anejo-margarita/...
Parsing https://vinepair.com/cocktail-recipe/the-sea-breeze-recipe/...
Parsing https://vinepair.com/cocktail-recipe/crow

In [170]:
len(recipe_links)

601